# Test: what is the best way to quantify the starburst age?
---

In this notebook:
* come up with a few different ways to quantify the "starburst age".
* write necessary functions to compute these quantities given a star formation history.
* Using MC runs (and later on regularisation too), see whether any of these quantities can reliably be measured.

Ideas:
* As a control: previous method - time index at which most recent star formation event drops to 0.
    * *potential issue*: definitely unreliable, given the "noise" in the recovered SFH that persists at even very high S/N.
* Mass weighted age below some age threshold.
    * *potential issue*: choice of age threshold is somewhat arbitrary.
* Light weighted age below some age threshold.
    * *potential issue*: same as above.
* Light/mass weighted age computed in a series of ranges
	* to be used in conjunction with the total mass fraction in each range, which will indicate whether there is a significant amount of SF. 
    * *potential issue*: again, choice of ranges is somewhat arbitrary.
* Counting backwards from $t = 0$, the time index at which the galaxy has built up $X \%$ of its total stellar mass. 
    * *potential issue*: may not be a good proxy for the precise quantity we're looking for.
* In the *most recent* star formation event, find the earliest time index at which the mass in each bin exceeds some minimum value, e.g., 1e7 solar masses or 0.01% of the total stellar mass.
    * *potential issue*: bins are not linearly spaced in age - how to deal with this?
* In the *most recent* star formation event, find the earliest time index at which the SFR exceeds some minimum value, e.g. $1 \rm \, M_\odot \, yr^{-1}$.
    * *potential issue*: this may not represent the actual SFR in the galaxy, if the new stars were obtained via a merger, for instance. 


In [3]:
%matplotlib widget

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [1]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, convert_mass_weights_to_light_weights
from ppxftests.sfhutils import compute_mw_age, compute_lw_age, compute_sfr_thresh_age, compute_sb_zero_age, compute_mass
from ppxftests.ppxf_plot import ppxf_plot, plot_sfh_mass_weighted, plot_sfh_light_weighted

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

plt.ion()
plt.close("all")

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"

from IPython.core.debugger import Tracer

In [5]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
z = 0.01

# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)


## How do the mass/light-weighted mean ages change as a function of "cutoff" age?
---
For a variety of different input SFHs, compute the light/mass-weighted mean age as a function of upper cutoff age.
Plot the MW/LW age as a function of "cutoff" age, with the 1 Gyr value shown for reference.

In [6]:
gal = 10
sfh_mw, sfh_lw, sfr_2D, sigma_gas_kms = load_sfh(gal, plotit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
# Load the DataFrame containing fitted SFHs for a subset of Phil's galaxies
df_fits = pd.read_hdf("/priv/meggs3/u5708159/ppxftests/ppxf_output.hd5")

In [47]:
df_fits = df_fits.set_index("ID")

In [50]:
gal = 0
# Compute vectors storing the mass- and light-weighted ages as a function of threshold age for the regularised and MC best-fit SFHs
# Load SFH
sfh_lw_MC = df_fits.loc[gal, "SFH - light weighted (MC, mean)"]
sfh_lw_regul = df_fits.loc[gal, "SFH - light weighted (regularised)"]

# Compute weighted ages
ages_lw_MC = [10**compute_lw_age(sfh_lw_MC, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0] for age_thresh_upper in ages[1:]]
ages_lw_regul = [10**compute_lw_age(sfh_lw_regul, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0] for age_thresh_upper in ages[1:]]

df_fits.loc[gal, "Mass weighted age vs. age threshold (MC, mean)"] = np.array(ages_lw_MC)
df_fits.loc[gal, "Mass weighted age vs. age threshold (regularised)"] = np.array(ages_lw_regul)

ValueError: Must have equal len keys and value when setting with an iterable

In [77]:
# Compute "truth" values
df = pd.DataFrame()

for gal in tqdm(range(100)):
    # Load SFH
    sfh_mw, sfh_lw, sfr_2D, sigma_gas_kms = load_sfh(gal, plotit=False)

    # Compute weighted ages
    for age_thresh_upper in ages[1:]:
        age_lw = 10**compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0]
        age_mw = 10**compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0]
        mass = compute_mass(sfh_mw, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)

        # Add to DataFrame
        df = df.append({
            "ID": gal,
            "Age threshold (upper, yr)": age_thresh_upper,
            "Mass-weighted mean age (yr)": age_mw,
            "Light-weighted mean age (yr)": age_lw,
            "Cumulative mass (Msun)": mass,
        }, ignore_index=True)

# df = df.set_index("Age threshold (upper, yr)")
    

In [157]:
gals = [int(g) for g in df_fits.index.values if g in df["ID"].values]
pp = PdfPages(os.path.join(fig_path, "age_mass_mesaures.pdf"))
for gal in tqdm(gals):
    # Plot
    plt.close("all")

    # Get sub-DataFrame for the ppxf fits for this galaxy
    df_gal = df[df["ID"] == gal]
    df_gal = df_gal.set_index("Age threshold (upper, yr)")

    # Get the measured SFHs
    sfh_lw_MC = df_fits.loc[gal, "SFH - light weighted (MC, mean)"]
    sfh_lw_regul = df_fits.loc[gal, "SFH - light weighted (regularised)"]
    sfh_mw_MC = df_fits.loc[gal, "SFH - mass weighted (MC, mean)"]
    sfh_mw_regul = df_fits.loc[gal, "SFH - mass weighted (regularised)"]

    # Compute weighted ages
    ages_lw_MC = [10**compute_lw_age(sfh_lw_MC, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0] for age_thresh_upper in ages[1:]]
    ages_lw_regul = [10**compute_lw_age(sfh_lw_regul, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0] for age_thresh_upper in ages[1:]]
    ages_mw_MC = [10**compute_mw_age(sfh_mw_MC, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0] for age_thresh_upper in ages[1:]]
    ages_mw_regul = [10**compute_mw_age(sfh_mw_regul, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper)[0] for age_thresh_upper in ages[1:]]

    # Compute weighted ages
    masses_lw_MC = [compute_mass(sfh_mw_MC, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper) for age_thresh_upper in ages[1:]]
    masses_lw_regul = [compute_mass(sfh_mw_regul, isochrones=isochrones, age_thresh_lower=None, age_thresh_upper=age_thresh_upper) for age_thresh_upper in ages[1:]]

    ###################################################################
    # Plot the SFH
    ###################################################################
    sfh_mw, sfh_lw, sfr_2D, sigma_gas_kms = load_sfh(gal, plotit=False)
    M_tot = np.nansum(sfh_mw)
    # plot_sfh_mass_weighted(sfh_mw, ages, metallicities)

    fig = plt.figure(figsize=(16, 10))
    ax1 = fig.add_axes([0.05, 0.575, 0.8, 0.4])
    ax2 = fig.add_axes([0.05, 0.1, 0.35, 0.4])
    ax3 = fig.add_axes([0.5, 0.1, 0.35, 0.4])
    axs = [ax1, ax2, ax3]

    axs[0].step(x=ages, y=np.nansum(sfh_mw, axis=0) / M_tot, where="mid", color="blue", label="SFH (input)", linewidth=2.5)
    axs[0].step(x=ages, y=np.nansum(sfh_mw_regul, axis=0) / M_tot, where="mid", color="indigo", label="SFH (regularised)", linewidth=1.0)
    axs[0].step(x=ages, y=np.nansum(sfh_mw_MC, axis=0) / M_tot, where="mid", color="lightblue", label="SFH (MC)", linewidth=1.0)

    axs[0].axhline(1e-4, color="k", ls="--", linewidth=1)
    axs[0].set_yscale("log")
    axs[0].set_xscale("log")
    axs[0].legend(loc="upper left", fontsize="x-small")
    axs[0].grid()
    axs[0].set_title(f"Galaxy ID {gal:004}")
    axs[0].set_ylabel("Mass fraction")
    axs[0].set_xlabel("Age (yr)")
    axs[0].autoscale(axis="x", tight=True, enable=True)

    ###################################################################
    # Plot the mean mass- and light-weighted age vs. age threshold
    ###################################################################
    # Plot the "true" values
    for weighttype, c in zip(["Mass", "Light"], ["green", "red"]):
        axs[1].step(x=df_gal.index, y=df_gal[f"{weighttype}-weighted mean age (yr)"], label=f"{weighttype}-weighted age", where="mid", linewidth=2.5, color=c)

    # Plot the values measured from the ppxf runs
    axs[1].step(x=ages[1:], y=ages_lw_regul, label=f"Measured light-weighted age (regularised)", where="mid", color="maroon")
    axs[1].step(x=ages[1:], y=ages_lw_MC, label=f"Measured light-weighted age (MC)", where="mid", color="orange")
    axs[1].step(x=ages[1:], y=ages_mw_regul, label=f"Measured mass-weighted age (regularised)", where="mid", color="darkgreen")
    axs[1].step(x=ages[1:], y=ages_mw_MC, label=f"Measured mass-weighted age (MC)", where="mid", color="lightgreen")

    # Decorations    
    axs[1].axhline(df_gal.loc[1.00e9, "Mass-weighted mean age (yr)"], color="k", ls="--", label="Mass-weighted mean age (< 1 Gyr)")
    axs[1].axhline(df_gal.loc[1.00e9, "Light-weighted mean age (yr)"], color="k", ls="-", label="Light-weighted mean age (< 1 Gyr)")
    axs[1].axhline(df_gal.loc[1.00e8, "Mass-weighted mean age (yr)"], color="grey", ls="--", label="Mass-weighted mean age (< 100 Myr)")
    axs[1].axhline(df_gal.loc[1.00e8, "Light-weighted mean age (yr)"], color="grey", ls="-", label="Light-weighted mean age (< 100 Myr)")
    axs[1].set_xscale("log")
    axs[1].set_yscale("log")
    axs[1].legend(loc="upper left", fontsize="x-small")
    axs[1].set_xlabel("Age threshold (yr)")
    axs[1].set_ylabel("Weighted mean age below threshold (yr)")
    axs[1].set_xlim([ages[0], ages[-1]])
    axs[1].grid()

    ###################################################################
    # Also plot cumulative mass expressed as a % so we can see what the threshold is at this S/N 
    ###################################################################
    # Plot the "true" values
    axs[2].step(x=df_gal.index, y=df_gal[f"Cumulative mass (Msun)"] / M_tot, label=f"Cumulative mass", where="mid", linewidth=2.5, color="blue")

    # Plot 
    axs[2].step(x=ages[1:], y=masses_lw_regul / M_tot, label=f"Cumulative mass (regularised)", where="mid", color="indigo")
    axs[2].step(x=ages[1:], y=masses_lw_MC / M_tot, label=f"Cumulative mass (MC)", where="mid", color="lightblue")

    # Decorations    
    axs[2].axhline(1e-4, color="k", ls="--", linewidth=1)
    axs[2].set_xscale("log")
    axs[2].set_yscale("log")
    axs[2].legend(loc="upper left", fontsize="x-small")
    axs[2].set_xlabel("Age (yr)")
    axs[2].set_ylabel("Cumulative mass fraction")
    axs[2].set_xlim([ages[0], ages[-1]])
    axs[2].grid()

    gal += 1

    pp.savefig(fig, bbox_inches="tight")
    
pp.close()

/home/u5708159/python/Modules/ppxftests/sfhutils.py:292: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:231: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/sfhutils.py:292: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:231: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/sfhutils.py:292: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:231: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/sfhutils.py:292: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:231: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
pp.close()

## How accurately can we recover the mass-weighted mean age?
---
To do:
* plot the MW mean age 

In [6]:
####################################################################
# Plot to check each of these have worked
####################################################################
sfh_mw, sfh_lw, sfr_2D, sigma_gas_kms = load_sfh(41, plotit=True)
sfh_mw_1D = np.nansum(sfh_mw, axis=0)
sfh_lw_1D = np.nansum(sfh_lw, axis=0)
sfr_1D = np.nansum(sfr_2D, axis=0)
M_tot = np.nansum(sfh_mw)

# Parameters for age estimators
age_thresh = 1e8
sfr_thresh = 1

# Compute the mass- and light-weighted ages in 3 different time bins
age_thresh_vals = [ages[0], 100e6, 1000e6, ages[-1]]
log_age_idx_mw_list = []
log_age_idx_lw_list = []
m_list = []
for aa in range(len(age_thresh_vals) - 1):
    log_age_mw, log_age_idx_mw = compute_mw_age(sfh_mw, isochrones=isochrones,
                                                age_thresh_lower=age_thresh_vals[aa],
                                                age_thresh_upper=age_thresh_vals[aa + 1])
    log_age_lw, log_age_idx_lw = compute_lw_age(sfh_lw, isochrones=isochrones,
                                                age_thresh_lower=age_thresh_vals[aa],
                                                age_thresh_upper=age_thresh_vals[aa + 1])
    m = compute_mass(sfh_mw, isochrones=isochrones,
                     age_thresh_lower=age_thresh_vals[aa],
                     age_thresh_upper=age_thresh_vals[aa + 1])
    log_age_idx_mw_list.append(log_age_idx_mw)
    log_age_idx_lw_list.append(log_age_idx_lw)
    m_list.append(m)
    
log_age_sb, log_age_idx_sb = compute_sb_zero_age(sfh_mw, isochrones=isochrones)  # "Starburst" age
log_age_sfr, log_age_idx_sfr = compute_sfr_thresh_age(sfh_mw, sfr_thresh, isochrones=isochrones)  # SFR threshold

print(f"compute_sb_zero_age(): {10**log_age_sb / 1e6:.2f} Myr")
print(f"compute_mw_age(): {10**log_age_mw / 1e6:.2f} Myr")
print(f"compute_lw_age(): {10**log_age_lw / 1e6:.2f} Myr")
print(f"compute_sfr_thresh_age(): {10**log_age_sfr / 1e6:.2f} Myr")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

compute_sb_zero_age(): 28.18 Myr
compute_mw_age(): 7149.67 Myr
compute_lw_age(): 5804.59 Myr
compute_sfr_thresh_age(): 25.12 Myr


In [54]:
####################################################################
# Plot the mass-weighted SFH and the cumulative mass counting from t = 0
####################################################################
fig, ax = plt.subplots(figsize=(13, 4))
fig.subplots_adjust(bottom=0.3, top=0.9)

# Plot the SFH and the cumulative mass from t = 0
ax.step(x=range(N_ages), y=sfh_mw_1D / M_tot, color="black", where="mid", label="1D SFH")
ax.step(x=range(N_ages), y=np.cumsum(sfh_mw_1D) / M_tot, where="mid", label="Cumulative mass", linewidth=0.5)

# Indicate each SB age measure
ax.axvline(log_age_idx_sb, color="grey", label="Starburst age")
for aa in range(len(log_age_idx_mw_list)):
    ax.axvline(log_age_idx_mw_list[aa], 
               color="blue", 
               label=f"Mass-weighted mean age ({age_thresh_vals[aa] / 1e6:.2f} ≤ t < {age_thresh_vals[aa + 1] / 1e6:.2f} Myr), log M = {np.log10(m_list[aa]):.2f}")
    ax.axvline(log_age_idx_lw_list[aa], 
               color="orange", 
               label=f"Light-weighted mean age ({age_thresh_vals[aa] / 1e6:.2f} ≤ t < {age_thresh_vals[aa + 1] / 1e6:.2f} Myr), log M = {np.log10(m_list[aa]):.2f}")
ax.axvline(log_age_idx_sfr, color="green", label="SFR")

ax.set_yscale("log")
ax.set_xticks(range(N_ages))
ax.set_xticklabels(ages / 1e6, rotation="vertical", fontsize="x-small")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
ax.set_ylabel(r"Stellar mass fraction $M/M_{\rm tot}$")
ax.set_xlabel("Bin age (Myr)")

####################################################################
# Plot the light-weighted SFH
####################################################################
fig, ax = plt.subplots(figsize=(13, 4))
fig.subplots_adjust(bottom=0.3, top=0.9)

# Plot the SFH and the cumulative mass from t = 0
ax.step(x=range(N_ages), y=sfh_lw_1D, color="black", where="mid", label="1D SFH")

# Indicate each SB age measure
ax.axvline(log_age_idx_sb, color="grey", label="Starburst age")
for aa in range(len(log_age_idx_mw_list)):
    ax.axvline(log_age_idx_mw_list[aa], 
               color="blue", 
               label=f"Mass-weighted mean age ({age_thresh_vals[aa] / 1e6:.2f} ≤ t < {age_thresh_vals[aa + 1] / 1e6:.2f} Myr), log M = {np.log10(m_list[aa]):.2f}")
    ax.axvline(log_age_idx_lw_list[aa], 
               color="orange", 
               label=f"Light-weighted mean age ({age_thresh_vals[aa] / 1e6:.2f} ≤ t < {age_thresh_vals[aa + 1] / 1e6:.2f} Myr), log M = {np.log10(m_list[aa]):.2f}")
ax.axvline(log_age_idx_sfr, color="green", label="SFR")

ax.set_yscale("log")
ax.set_xticks(range(N_ages))
ax.set_xticklabels(ages / 1e6, rotation="vertical", fontsize="x-small")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
ax.set_ylabel(r"Light-weighted SFH (at 5000 Å)")
ax.set_xlabel("Bin age (Myr)")


####################################################################
# Plot the mean SFR in each bin
####################################################################
# Plot the mean SFR in each bin
fig, ax = plt.subplots(figsize=(13, 4))
fig.subplots_adjust(bottom=0.3, top=0.9)
ax.step(x=range(N_ages), y=sfr_1D, where="mid", label="Average SFR")

# Indicate each SB age measure
ax.axvline(log_age_idx_sb, color="grey", label="Starburst age")
ax.axvline(log_age_idx_mw, color="blue", label=f"Mass-weighted mean age (< {age_thresh / 1e6:.0f} Myr)")
ax.axvline(log_age_idx_lw, color="orange", label=f"light-weighted mean age (< {age_thresh / 1e6:.0f} Myr)")
ax.axvline(log_age_idx_sfr, color="green", label="SFR")

ax.axhline(sfr_thresh, linestyle="--", color="grey", label="SFR threshold")

ax.set_yscale("log")
ax.set_xticks(range(N_ages))
ax.set_xticklabels(ages / 1e6, rotation="vertical", fontsize="x-small")
ax.grid()
ax.legend(fontsize="x-small")
ax.autoscale(axis="x", tight=True, enable=True)
ax.set_ylabel(r"Mean SFR ($\rm M_\odot \, yr^{-1}$)")
ax.set_xlabel("Bin age (Myr)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Bin age (Myr)')

In [12]:
####################################################################
# Plot the mass-weighted mean age as a function of age threshold
####################################################################
mw_mean_ages = [compute_mw_age(sfh_mw, age)[0] for age in ages]
fig, ax = plt.subplots()
ax.scatter(ages, mw_mean_ages)
ax.set_xlabel("Age threshold (Myr)")
ax.set_ylabel("Mass-weighted mean age (Myr)")
ax.set_xscale("log")
ax.set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'sfh_1D' is not defined

## How accurately can ppxf return the mass- and luminosity-weighted ages in different bins?
---
Run ppxf on a few different input SFHs, using the MC method. 
Plot the SFH with the "SFR ages" indicated for both the input and the output.

In [49]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [145]:
###########################################################################
# Convenience functions for computing mean quantities from a list of ppxf instances 
###########################################################################
def compute_mean_1D_sfh(pp_list, weighttype):
    """
    Convenience function for computing the mean SFH given a list of ppxf
    instances.
    """
    assert weighttype == "lw" or weighttype == "mw",\
        "weighttype must be 'lw' or 'mw'!"

    if weighttype == "lw":
        sfh_list = [pp.sfh_lw_1D for pp in pp_list]
    elif weighttype == "mw":
        sfh_list = [pp.sfh_mw_1D for pp in pp_list]
    sfh_1D_mean = np.nansum(np.array(sfh_list), axis=0) / len(sfh_list)

    return sfh_1D_mean

def compute_mean_mass(pp_list, age_thresh_lower, age_thresh_upper):
    """
    Convenience function for computing the mean & std. dev. of the total mass
    in the range [age_thresh_lower, age_thresh_upper] given a list of ppxf instances.
    """
    sfh_list = [pp.weights_mass_weighted for pp in pp_list]
    mass_list = [compute_mass(sfh_mw, isochrones, age_thresh_lower, age_thresh_upper) for sfh_mw in sfh_list]
    mass_mean = np.nanmean(mass_list)
    mass_std = np.nanstd(mass_list)
    return mass_mean, mass_std


def compute_mean_sfr(pp_list):
    """
    Convenience function for computing the mean SFR given a list of ppxf
    instances.
    """
    sfr_list = [pp.sfr_mean for pp in pp_list]
    sfr_mean = np.nansum(np.array(sfr_list), axis=0) / len(sfr_list)
    return sfr_mean

def compute_mean_age(pp_list, weighttype, age_thresh_lower, age_thresh_upper):
    """
    Convenience function for computing the mean & std. dev. of the mass-
    weighted age in the range [age_thresh_lower, age_thresh_upper] given 
    a list of ppxf instances.
    """
    assert weighttype == "lw" or weighttype == "mw",\
        "weighttype must be 'lw' or 'mw'!"
    
    if weighttype == "mw":
        sfh_list = [pp.weights_mass_weighted for pp in pp_list]
        age_list = [10**compute_mw_age(sfh, isochrones, age_thresh_lower, age_thresh_upper)[0] for sfh in sfh_list]
        age_mean = np.nanmean(age_list)
        age_std = np.nanstd(age_list)
        
    elif weighttype == "lw":
        sfh_list = [pp.weights_light_weighted for pp in pp_list]
        age_list = [10**compute_lw_age(sfh, isochrones, age_thresh_lower, age_thresh_upper)[0] for sfh in sfh_list]
        age_mean = np.nanmean(age_list)
        age_std = np.nanstd(age_list)
    
    return age_mean, age_std

def compute_mean_sfr_thresh_age(pp_list, sfr_thresh):
    """
    Convenience function for computing the mean and std. dev. in the
    SFR threshold age from a list of ppxf instances.
    """
    sfr_age_list = [10**compute_sfr_thresh_age(pp.weights_mass_weighted, sfr_thresh, isochrones)[0] for pp in pp_list]
    sfr_age_mean = np.nanmean(sfr_age_list)
    sfr_age_std = np.nanstd(sfr_age_list)

    return sfr_age_mean, sfr_age_std


In [158]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
sigma_star_kms = 250
SNR = 100
z = 0.01

niters = 100
nthreads = 20

# For computing mean ages, etc.
age_thresh_vals = [None, 1e7, 1e8, 1e9, None]
sfr_thresh = 1

# DataFrame for storing results 
df = pd.DataFrame(index=gals)
df.index.name = "ID"


In [165]:
df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (truth)"]

12588999.999999993

In [215]:
###########################################################################
# Define the SFH
###########################################################################
gal = 28

sfh_mw_input, sfh_lw_input, sfr_avg_input = load_sfh(gal=gal, plotit=True)
sfh_mw_1D_input = np.nansum(sfh_mw_input, axis=0)
sfh_lw_1D_input = np.nansum(sfh_lw_input, axis=0)

# Compute truth values 
age_sfr_input = 10**compute_sfr_thresh_age(sfh_mw, sfr_thresh, isochrones=isochrones)[0]  # SFR threshold
age_sb_input = 10**compute_sb_zero_age(sfh_mw, isochrones=isochrones)[0]  # "Starburst" age
df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (truth)"] = age_sfr_input
df.loc[gal, f"SB age (truth)"] = age_sb_input

for aa in range(len(age_thresh_vals)- 1):
    age_thresh_lower = age_thresh_vals[aa]
    age_thresh_upper = age_thresh_vals[aa + 1]

    # Determine age boundaries
    if age_thresh_lower is None:
        age_thresh_lower = ages[0]
    if age_thresh_upper is None:
        age_thresh_upper = ages[-1]
    
    # Compute mass- and light-weighted mean ages, and the total mass too
    age_mw_input = 10**compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    age_lw_input = 10**compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    mass = compute_mass(sfh_mw, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)
    
    # Store in DataFrame
    df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (truth)"] = age_mw_input
    df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (truth)"] = age_lw_input
    df.loc[gal, f"Mass {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (truth)"] = mass


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [212]:
np.log10(df[[c for c in df.columns if "Mass" in c]])

,Mass 6.60 < log t < 7.00 (truth),Mass 7.00 < log t < 8.00 (truth),Mass 8.00 < log t < 9.00 (truth),Mass 9.00 < log t < 10.25 (truth),Mass 6.60 < log t < 7.00 (MC) mean,Mass 6.60 < log t < 7.00 (MC) std. dev.,Mass 7.00 < log t < 8.00 (MC) mean,Mass 7.00 < log t < 8.00 (MC) std. dev.,Mass 8.00 < log t < 9.00 (MC) mean,Mass 8.00 < log t < 9.00 (MC) std. dev.,Mass 9.00 < log t < 10.25 (MC) mean,Mass 9.00 < log t < 10.25 (MC) std. dev.,Mass 6.60 < log t < 7.00 (regul),Mass 7.00 < log t < 8.00 (regul),Mass 8.00 < log t < 9.00 (regul),Mass 9.00 < log t < 10.25 (regul)
ID,,,,,,,,,,,,,,,,
0,-inf,8.080792,8.799893,11.193843,7.971063,7.10108,7.642605,8.047643,9.17385,8.713251,11.746359,10.591538,7.065157,8.528689,8.842562,11.690287
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
###########################################################################
# Create spectrum
###########################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

/home/u5708159/python/Modules/ppxftests/mockspec.py:179: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/ppxftests/mockspec.py:399: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [155]:
###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach
###########################################################################
# Input arguments
seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Elapsed time in ppxf: {time() - t:.2f} s")

Running ppxf on 20 threads...



Elapsed time in ppxf: 200.80 s


In [176]:
###########################################################################
# Run ppxf with regularisation
###########################################################################
t = time()
pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="auto",
              isochrones=isochrones,
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.18 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.4052...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 67.71 s
Iteration 1: optimal regul = 10000.00; Δm = 5.74407e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 58.262
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 65.77 s
Iteration 2: optimal regul = 20000.00; Δm = 3.04285e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 27.281
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 73.12 s
Iteration 3: optimal regul = 26500.00; Δm = 1.31489e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 1.1

In [210]:
###########################################################################
# Compute quantities from the regularised fit
###########################################################################
# Get the SFH and SFR
sfh_regul_mw_1D = pp_regul.sfh_mw_1D
sfh_regul_lw_1D = pp_regul.sfh_lw_1D
sfr_avg_regul = pp_regul.sfr_mean

age_sfr_regul = 10**compute_sfr_thresh_age(sfh_regul_lw_1D, sfr_thresh, isochrones=isochrones)[0]  # SFR threshold
age_sb_regul = 10**compute_sb_zero_age(sfh_regul_lw_1D, isochrones=isochrones)[0]  # "Starburst" age
df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (regul)"] = age_sfr_regul
df.loc[gal, f"SB age (regul)"] = age_sb_regul

for aa in range(len(age_thresh_vals)- 1):
    age_thresh_lower = age_thresh_vals[aa]
    age_thresh_upper = age_thresh_vals[aa + 1]

    # Determine age boundaries
    if age_thresh_lower is None:
        age_thresh_lower = ages[0]
    if age_thresh_upper is None:
        age_thresh_upper = ages[-1]
    
    # Compute mass- and light-weighted mean ages, and the total mass too
    age_mw_regul = 10**compute_mw_age(sfh_regul_lw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    age_lw_regul = 10**compute_lw_age(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    mass_regul = compute_mass(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)
    
    # Store in DataFrame
    df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (regul)"] = age_mw_regul
    df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (regul)"] = age_lw_regul
    df.loc[gal, f"Mass {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (regul)"] = mass_regul

In [191]:
###########################################################################
# Compute average quantities from the MC simulations
###########################################################################
# Compute the mean SFH and SFR from the lists of MC runs
sfh_MC_lw_1D_mean = compute_mean_1D_sfh(pp_list, "lw")
sfh_MC_mw_1D_mean = compute_mean_1D_sfh(pp_list, "mw")
sfr_avg_MC = compute_mean_sfr(pp_list)

# Compute the "SFR age"
age_sfr_mean, age_sfr_std = compute_mean_sfr_thresh_age(pp_list, sfr_thresh)
df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (MC) mean"] = age_sfr_mean
df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (MC) std. dev."] = age_sfr_std

# Compute the mean mass- and light-weighted ages plus the total mass in a series of age ranges
for aa in range(len(age_thresh_vals) - 1):
    age_thresh_lower = age_thresh_vals[aa]
    age_thresh_upper = age_thresh_vals[aa + 1]
    
    if age_thresh_lower is None:
        age_thresh_lower = ages[0]
    if age_thresh_upper is None:
        age_thresh_upper = ages[-1]
        
    # Compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_lw_mean, age_lw_std = compute_mean_age(pp_list, "lw", age_thresh_lower, age_thresh_upper)
    age_mw_mean, age_mw_std = compute_mean_age(pp_list, "mw", age_thresh_lower, age_thresh_upper)
    mass_mean, mass_std = compute_mean_mass(pp_list, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

    # Put in DataFrame
    df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) mean"] = age_mw_mean
    df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) mean"] = age_lw_mean
    df.loc[gal, f"Mass {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) mean"] = mass_mean
    df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) std. dev."] = age_mw_std
    df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) std. dev."] = age_lw_std
    df.loc[gal, f"Mass {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) std. dev."] = mass_std

/home/u5708159/python/Modules/ppxftests/sfhutils.py:226: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:193: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


In [205]:
###########################################################################
# Plot the input mass- and light-weighted SFHs
###########################################################################
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 15))
fig.subplots_adjust(hspace=0.35)
log_scale = True
info_str = r"$S/N = %d, z = %.3f, \sigma_* = %d\rm\, km\,s^{-1}$" % (SNR, z, sigma_star_kms)
for ax, weighttype in zip(axs, ["mw", "lw", "sfr"]):
    # Plot the SFHs from each ppxf run, plus the "truth" SFH
    if weighttype == "mw":
        ax.set_title(f"Galaxy {gal:004}: mass-weighted template weights ({info_str})")
        ax.fill_between(ages, sfh_mw_1D_input, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
        ax.step(ages, sfh_MC_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)", alpha=0.5)
        ax.step(ages, sfh_regul_mw_1D, color="green", where="mid", label="Mean ppxf fit (regularised fit)", alpha=0.5)
        ax.set_ylim([1e3, None])
        ax.set_ylabel(r"MW template weight ($\rm M_\odot$)")
    elif weighttype == "lw":
        ax.set_title(f"Galaxy {gal:004}: light-weighted template weights ({info_str})")
        ax.fill_between(ages, sfh_lw_1D_input, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
        ax.step(ages, sfh_MC_lw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)", alpha=0.5)
        ax.step(ages, sfh_regul_lw_1D, color="green", where="mid", label="Mean ppxf fit (regularised fit)", alpha=0.5)
        ax.set_ylim([1e35, None])
        ax.set_ylabel(r"LW template weight ($\rm erg\,s^{-1}\,Å^{-1}$)")
    elif weighttype == "sfr":
        ax.set_title(f"Galaxy {gal:004}: mean SFR ({info_str})")
        ax.fill_between(ages, sfr_avg_input, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
        ax.step(ages, sfr_avg_MC, color="red", where="mid", label="Mean ppxf fit (MC simulations)", alpha=0.5)
        ax.step(ages, sfr_avg_regul, color="green", where="mid", label="Mean ppxf fit (regularised fit)", alpha=0.5)
        ax.set_ylim([1e-2, None])
        ax.set_ylabel(r"Mean SFR ($\rm M_\odot\,yr^{-1}$)")

    # Plot horizontal error bars indicating the SFR threshold age from the MC simulations
    y1, y2 = ax.get_ylim()
    y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
    ax.errorbar(x=df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (MC) mean"],
                xerr=df.loc[gal, f"SFR age (> {sfr_thresh} Msun yr^-1) (MC) std. dev."],
                y=y, 
                marker="*", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10, markersize=10,
                label="SFR age (mean, MC simulations)")
    ax.errorbar(x=df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (truth)"], xerr=0, markersize=10,
                y=y, 
                marker="*", mfc="lightblue", mec="blue", ecolor="lightblue", linestyle="none",
                label="SFR age (input)")
    ax.errorbar(x=df.loc[gal, f"SFR age (>{sfr_thresh} Msun yr^-1) (regul)"], xerr=0, markersize=10,
                y=y, 
                marker="*", mfc="lightgreen", mec="green", ecolor="green", linestyle="none",
                label="SFR age (regularised fit)")

    # Plot horizontal error bars indicating the mean mass- and light-weighted ages ages from the MC simulations
    for aa in range(len(age_thresh_vals) - 1):
        age_thresh_lower = age_thresh_vals[aa]
        age_thresh_upper = age_thresh_vals[aa + 1]

        if age_thresh_lower is None:
            age_thresh_lower = ages[0]
        if age_thresh_upper is None:
            age_thresh_upper = ages[-1]
        
        # mass-weighted age
        y = 10**(0.8 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.8 * y2
        ax.errorbar(x=df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) mean"],
                    xerr=df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) std. dev."],
                    y=y,
                    marker="D", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="Mean MW age in range (MC simulations)" if aa == 0 else None)
        ax.errorbar(x=df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (regul)"], xerr=0,
                    y=y,
                    marker="D", mfc="lightgreen", mec="green", ecolor="green", linestyle="none",
                    label="Mean MW age in range (regularised fit)" if aa == 0 else None)
        ax.errorbar(x=df.loc[gal, f"MW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (truth)"], xerr=0,
                    y=y,
                    marker="D", mfc="lightblue", mec="blue", ecolor="lightblue", linestyle="none",
                    label="Mean MW age in range (input)" if aa == 0 else None)

        # light-weighted age
        y = 10**(0.7 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.7 * y2
        ax.errorbar(x=df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) mean"],
                    xerr=df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (MC) std. dev."],
                    y=y,
                    marker="X", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="Mean LW age in range (MC simulations)" if aa == 0 else None)
        ax.errorbar(x=df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (regul)"], xerr=0,
                    y=y,
                    marker="X", mfc="lightgreen", mec="green", ecolor="green", linestyle="none",
                    label="Mean LW age in range (regularised fit)" if aa == 0 else None)
        ax.errorbar(x=df.loc[gal, f"LW age {np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f} (truth)"], xerr=0,
                    y=y,
                    marker="X", mfc="lightblue", mec="blue", ecolor="lightblue", linestyle="none",
                    label="Mean LW age in range (input)" if aa == 0 else None)

        ax.axvline(age_thresh_lower, color="black", linestyle="--", label="Age range" if aa == 0 else None)
        ax.axvline(age_thresh_upper, color="black", linestyle="--")

    # Decorations 
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_xlabel("Age (Myr)")
    ax.legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
    ax.set_yscale("log") if log_scale else None
    ax.set_xscale("log")
    ax.grid()
    
fig.savefig(os.path.join(fig_path, "basic_tests", f"ga{gal:004}.pdf", format="pdf", bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [206]:
df

,SFR age (>1 Msun yr^-1) (truth),SB age (truth),MW age 6.60 < log t < 7.00 (truth),LW age 6.60 < log t < 7.00 (truth),Mass 6.60 < log t < 7.00 (truth),MW age 7.00 < log t < 8.00 (truth),LW age 7.00 < log t < 8.00 (truth),Mass 7.00 < log t < 8.00 (truth),MW age 8.00 < log t < 9.00 (truth),LW age 8.00 < log t < 9.00 (truth),...,Mass 6.60 < log t < 7.00 (regul),MW age 7.00 < log t < 8.00 (regul),LW age 7.00 < log t < 8.00 (regul),Mass 7.00 < log t < 8.00 (regul),MW age 8.00 < log t < 9.00 (regul),LW age 8.00 < log t < 9.00 (regul),Mass 8.00 < log t < 9.00 (regul),MW age 9.00 < log t < 10.25 (regul),LW age 9.00 < log t < 10.25 (regul),Mass 9.00 < log t < 10.25 (regul)
ID,,,,,,,,,,,,,,,,,,,,,
0,12589000.0,14125000.0,NaN,NaN,0.0,4.468414e+07,3.369922e+07,120445950.0,4.389734e+08,3.013682e+08,...,3.032384e+38,2.934882e+07,3.734690e+07,2.105153e+39,2.422139e+08,3.254544e+08,1.122999e+39,7.661003e+09,9.822382e+09,2.956424e+40
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
